In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import tomli
import toml
import subprocess
import json

In [2]:
dataset = "bank_latent"

In [3]:
# This csv file will be the entry point from the variational auto encoder part
data = pd.read_csv("../../data/processed/bank,.csv")
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [4]:
idx = np.arange(0, X.shape[0])
train_idx, test_idx = train_test_split(idx, test_size =0.3, random_state=42)
val_idx, test_idx,  = train_test_split(test_idx, test_size =0.5, random_state=42)
X_train, X_val, X_test = X[train_idx], X[val_idx], X[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

In [5]:
train_size, valu_size, test_size = X_train.shape[0], X_val.shape[0], X_test.shape[0]
print(train_size, valu_size, test_size)

3500 750 750


In [6]:
np.save(f'../data/{dataset}/X_num_train.npy', X_train)
np.save(f'../data/{dataset}/X_num_val.npy', X_val)
np.save(f'../data/{dataset}/X_num_test.npy', X_test)

FileNotFoundError: [Errno 2] No such file or directory: '../data/bank_latent/X_num_train.npy'

In [ ]:
np.save(f'../data/{dataset}/y_train.npy', y_train)
np.save(f'../data/{dataset}/y_val.npy', y_val)
np.save(f'../data/{dataset}/y_test.npy', y_test)

In [7]:
num_feature_size, cat_features_size = 9, 0 # These are default considering latent size of each client is 3 in VAE

In [8]:
info_file = {
    "task_type": "binclass",
    "name": f"{dataset}",
    "id": f"{dataset}--id",
    "train_size": train_size,
    "val_size": valu_size,
    "test_size": test_size,
    "n_num_features": num_feature_size,
    "n_cat_features": cat_features_size
}

In [9]:
os.makedirs(f"../data/{dataset}", exist_ok=True)

In [10]:
with open(f"../data/{dataset}/info.json", "w") as info_writer:
    json.dump(info_file, info_writer)

In [11]:
os.makedirs(f"../exp/{dataset}", exist_ok=True)

In [12]:
config_file = {
    'seed': 0,
    'parent_dir': f'../exp/{dataset}/',
    'real_data_path': f'../data/{dataset}/',
    'model_type': 'mlp',
    'num_numerical_features': num_feature_size,   # Set the number of numerical features here
    'device': 'cuda:0',  
    'model_params': {       # Change the denoising architecture here as per your liking
        'd_in': 8,
        'num_classes': 2,
        'is_y_cond': True,
        'rtdl_params': {
            'd_layers': [
                128,
                512
            ],
            'dropout': 0.0
        }
    },
    'diffusion_params': {
        'num_timesteps': 1000,
        'gaussian_loss_type': 'mse'
    },
    'train': {
        'main': {
            'steps': 30000,
            'lr': 1.1510940031144828e-05,
            'weight_decay': 0.0,
            'batch_size': 4096
        },
        'T': {
            'seed': 0,
            'normalization': 'quantile',
            'num_nan_policy': '__none__',
            'cat_nan_policy': '__none__',
            'cat_min_frequency': '__none__',
            'cat_encoding': '__none__',
            'y_policy': 'default'
        }
    },
    'sample': {
        'num_samples': 768,
        'batch_size': 500,
        'seed': 0
    },
    'eval': {
        'type': {
            'eval_model': 'catboost',
            'eval_type': 'synthetic'
        },
        'T': {
            'seed': 0,
            'normalization': '__none__',
            'num_nan_policy': '__none__',
            'cat_nan_policy': '__none__',
            'cat_min_frequency': '__none__',
            'cat_encoding': '__none__',
            'y_policy': 'default'
        }
    }
}

In [13]:
with open(f"../exp/{dataset}/config.toml", 'w') as toml_file:
    toml.dump(config_file, toml_file)

In [14]:
command = [
    "python", 
    "pipeline.py", 
    "--config", f"../exp/{dataset}/config.toml", 
    "--train", 
    "--sample"
]
result = subprocess.run(command, capture_output=True, text=True)

In [15]:
result.stderr

'Traceback (most recent call last):\n  File "/Users/arnobchowdhury/Documents/university_work/Thesis/Silofuse/central_backbone/scripts/pipeline.py", line 5, in <module>\n    from train import train\n  File "/Users/arnobchowdhury/Documents/university_work/Thesis/Silofuse/central_backbone/scripts/train.py", line 5, in <module>\n    import zero\nModuleNotFoundError: No module named \'zero\'\n'